# Feature Engineering and Model Training

This notebook covers feature engineering, model training, and evaluation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

In [ ]:
df = pd.read_csv('/mnt/data/dataset.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

## Basic Feature Cleaning

In [ ]:
df = df.drop_duplicates()
df.isnull().sum()

## Handling Missing Values

In [ ]:
for col in df.select_dtypes(include=np.number).columns:
    df[col].fillna(df[col].median(), inplace=True)
for col in df.select_dtypes(exclude=np.number).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

## Feature Engineering

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns
for col in num_cols:
    df[f'{col}_squared'] = df[col] ** 2

df.head()

In [ ]:
df['row_sum'] = df[num_cols].sum(axis=1)

In [ ]:
df['row_mean'] = df[num_cols].mean(axis=1)

## Encoding Categorical Features

In [ ]:
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()

## Feature-Target Split

In [ ]:
X = df_encoded.iloc[:, :-1]
y = df_encoded.iloc[:, -1]
X.shape, y.shape

## Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

## Feature Scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Linear Regression Model

In [ ]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

In [ ]:
y_pred_lr = lr.predict(X_test_scaled)
print('MSE:', mean_squared_error(y_test, y_pred_lr))
print('R2:', r2_score(y_test, y_pred_lr))

## Random Forest Regressor

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf.predict(X_test)
print('MSE:', mean_squared_error(y_test, y_pred_rf))
print('R2:', r2_score(y_test, y_pred_rf))

## Feature Importance

In [ ]:
importances = pd.Series(rf.feature_importances_, index=X.columns)
importances.sort_values(ascending=False).head(10)

## Model Comparison

In [ ]:
results = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest'],
    'MSE': [mean_squared_error(y_test, y_pred_lr), mean_squared_error(y_test, y_pred_rf)],
    'R2': [r2_score(y_test, y_pred_lr), r2_score(y_test, y_pred_rf)]
})
results

## Save Trained Model

In [ ]:
import joblib
joblib.dump(rf, '/mnt/data/rf_model.pkl')